In [ ]:
cd ../../..

In [ ]:
%load_ext dotenv
%dotenv -o config/secrets/.env.linkedin.local

In [ ]:
from linkedin_easy_apply_bot.backend import config
from linkedin_easy_apply_bot.backend.main import EasyApplyBot

In [ ]:

assert config.LINKEDIN_USERNAME is not None
assert config.LINKEDIN_PASSWORD is not None

In [ ]:
from linkedin_easy_apply_bot.backend.utils import yaml_utils
parameters = yaml_utils.read_file('src/config.yaml')
parameters

In [ ]:
import time
import random
from linkedin_easy_apply_bot.backend.utils import logging_utils


log = logging_utils.get_logger(__name__)

def start_apply(positions, locations) -> None:
        start: float = time.time()
        combos: list = []
        while len(combos) < len(positions) * len(locations):
            position = positions[random.randint(0, len(positions) - 1)]
            location = locations[random.randint(0, len(locations) - 1)]
            combo: tuple = (position, location)
            if combo not in combos:
                combos.append(combo)
                log.info(f"Applying to {position}: {location}")
                location = "&location=" + location
                yield (position, location)
            if len(combos) > 500:
                break

In [ ]:
position_location_pairs = list(start_apply(parameters["positions"], parameters["locations"]))
position_location_pairs

In [ ]:
from datetime import datetime
import re
from linkedin_easy_apply_bot.backend.utils import json_utils

job_ids = {}
def create_new_job(job_id):

    if job_id not in job_ids:

        job_ids[job_id] = {
            "is_processed": False
        }
    
    return job_ids[job_id]


def update_job(job_id, button, browserTitle, salary, has_applied = False, file_path = 'files/data/jobs.csv',):

    if job_id in job_ids:
        job_ids[job_id]["is_processed"] = True

    def re_extract(text, pattern):
        target = re.search(pattern, text)
        if target:
            target = target.group(1)
        return target

    job_ids[job_id]['timestamp'] = datetime.now().isoformat()
    job_ids[job_id]['can_easy_apply'] = False if button == False else True
    job_ids[job_id]['job'] = re_extract(browserTitle.split(" | ")[0], r"\(?\d?\)?\s?(\w.*)")
    job_ids[job_id]['company'] = re_extract(browserTitle.split(" | ")[1], r"(\w.*)")
    job_ids[job_id]['salary'] = salary
    
    

    json_utils.data_to_json('files/data/jobs.json', job_ids)



In [ ]:
def discover_new_jobs(bot, position, location):
        count_application = 0
        count_job = 0
        jobs_per_page = 0
        start_time: float = time.time()

        log.info("Looking for jobs.. Please wait..")
        bot.browser.set_window_position(1, 1)
        bot.browser.maximize_window()
        bot.browser, _ = bot.next_jobs_page(
            position, location, jobs_per_page, experience_level=bot.experience_level
        )
        log.info("Looking for jobs.. Please wait..")

        while time.time() - start_time < bot.MAX_SEARCH_TIME:
            try:
                log.info(
                    f"{(bot.MAX_SEARCH_TIME - (time.time() - start_time)) // 60} minutes left in this search"
                )

                # sleep to make sure everything loads, add random to make us look human.
                randoTime: float = random.uniform(1.5, 2.9)
                log.debug(f"Sleeping for {round(randoTime, 1)}")
                # time.sleep(randoTime)
                bot.load_page(sleep=0.5)

                # LinkedIn displays the search results in a scrollable <div> on the left side, we have to scroll to its bottom

                # scroll to bottom

                if bot.is_present(bot.locator["search"]):
                    scrollresults = bot.get_elements("search")
                    #     bot.browser.find_element(By.CLASS_NAME,
                    #     "jobs-search-results-list"
                    # )
                    # Selenium only detects visible elements; if we scroll to the bottom too fast, only 8-9 results will be loaded into IDs list
                    for i in range(300, 3000, 100):
                        bot.browser.execute_script(
                            "arguments[0].scrollTo(0, {})".format(i), scrollresults[0]
                        )
                    scrollresults = bot.get_elements("search")
                    # time.sleep(1)

                # get job links, (the following are actually the job card objects)
                if bot.is_present(bot.locator["links"]):
                    links = bot.get_elements("links")
                    # links = bot.browser.find_elements("xpath",
                    #     '//div[@data-job-id]'
                    # )

                    # children selector is the container of the job cards on the left
                    for link in links:
                        if "Applied" not in link.text:  # checking if applied already
                            if (
                                link.text not in bot.blacklist
                            ):  # checking if blacklisted
                                job_id = link.get_attribute("data-job-id")
                                if job_id == "search":
                                    log.debug(
                                        "Job ID not found, search keyword found instead? {}".format(
                                            link.text
                                        )
                                    )
                                    continue
                                else:
                                    yield job_id

                    bot.browser, jobs_per_page = bot.next_jobs_page(
                        position,
                        location,
                        jobs_per_page,
                        experience_level=bot.experience_level,
                    )
                else:
                    bot.browser, jobs_per_page = bot.next_jobs_page(
                        position,
                        location,
                        jobs_per_page,
                        experience_level=bot.experience_level,
                    )

            except Exception as e:
                print(e)


In [ ]:
# def apply_to_job(bot, job_id, blackListTitles=[]) -> bool:
#         # #bot.avoid_lock() # annoying

#         # get job page
#         bot.get_job_page(job_id)

#         # let page load
#         time.sleep(1)

#         # get easy apply button
#         button = bot.get_easy_apply_button()

#         # word filter to skip positions not wanted
#         if button is not False:
#             if any(word in bot.browser.title for word in blackListTitles):
#                 log.info(
#                     "skipping this application, a blacklisted keyword was found in the job position"
#                 )
#                 string_easy = "* Contains blacklisted keyword"
#                 result = False
#             else:
#                 string_easy = "* has Easy Apply Button"
#                 log.info("Clicking the EASY apply button")
#                 button.click()
#                 clicked = True
#                 time.sleep(1)
#                 bot.fill_out_fields()
#                 result: bool = bot.send_resume()
#                 if result:
#                     string_easy = "*Applied: Sent Resume"
#                 else:
#                     string_easy = "*Did not apply: Failed to send Resume"
#         elif "You applied on" in bot.browser.page_source:
#             log.info("You have already applied to this position.")
#             string_easy = "* Already Applied"
#             result = False
#         else:
#             log.info("The Easy apply button does not exist.")
#             string_easy = "* Doesn't have Easy Apply Button"
#             result = False

#         # position_number: str = str(count_job + jobs_per_page)
#         log.info(f"\nPosition {job_id}:\n {bot.browser.title} \n {string_easy} \n")

#         bot.write_to_file(button, job_id, bot.browser.title, result)
#         return result

In [ ]:
def save_jobs(bot, job_id):
    print(job_ids)
    bot.get_job_page(job_id)
    time.sleep(1)
    button = bot.get_easy_apply_button()
    salary = bot.get_salary()
    page_title = bot.browser.title
    update_job(job_id,button, page_title, salary)
        


In [ ]:
bot = EasyApplyBot(
    username=config.LINKEDIN_USERNAME,
    password=config.LINKEDIN_PASSWORD
)


for position, location in position_location_pairs:
    for job_id in discover_new_jobs(bot, position, location):
        job = create_new_job(job_id)

        if not job['is_processed']:
            save_jobs(bot, job_id)

        

bot.browser.quit()
